## 卷积神经网络 & TensorBoard

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
import os

## 获取数据

In [2]:
# GIST_URL = 'https://gist.githubusercontent.com/dandelionmane/4f02ab8f1451e276fea1f165a20336f1/raw/dfb8ee95b010480d56a73f324aca480b3820c180'
# urllib.urlretrieve(GIST_URL + 'labels_1024.tsv', LOGDIR + 'labels_1024.tsv')
# urllib.urlretrieve(GIST_URL + 'sprite_1024.png', LOGDIR + 'sprite_1024.png')

In [3]:
mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## 定义模型

In [4]:
def weight_initial(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial, name='W')

def bias_initial(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name='b')

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1],strides=[1,2,2,1], padding='SAME')

def conv_layer(input, size_in, size_out, name='conv'):
    with tf.name_scope(name):
        w = weight_initial([5,5,size_in,size_out])
        b = bias_initial([size_out])
        h = tf.nn.relu(conv2d(input,w)+b)
        tf.summary.histogram('weights', w)
        tf.summary.histogram('biases', b)
        tf.summary.histogram('activation', h)
        return  max_pool_2x2(h)
    
def fc_layer(input, size_in, size_out, name='fc'):
    with tf.name_scope(name):
        w = weight_initial([size_in,size_out])
        b = bias_initial([size_out])
        h = tf.nn.relu(tf.matmul(input, w)+b)
        tf.summary.histogram('weights', w)
        tf.summary.histogram('biases', b)
        tf.summary.histogram('activation', h)
        return h

In [5]:
def make_hparam_string(learning_rate, use_two_fc, use_two_conv):
    conv_param = "conv=2" if use_two_conv else "conv=1"
    fc_param = "fc=2" if use_two_fc else "fc=1"
    return "lr_%.0E,%s,%s" % (learning_rate, conv_param, fc_param)

LOG_DIR = 'E:/workspace/documents_new/stu_github/MyStudy/ML/python/code/tensorboard/board003/'

In [6]:
def model(learning_rate, use_two_conv, use_two_fc, hparam):
    tf.reset_default_graph()
    sess = tf.Session()
    # 输入层
    x = tf.placeholder(tf.float32,shape=[None,784], name='input_x')
    x_image = tf.reshape(x, [-1,28,28,1])
    tf.summary.image('input',x_image,3)
    y = tf.placeholder(tf.float32,shape=[None,10], name='input_y')

    if use_two_conv:
        # 卷积层1
        conv1 = conv_layer(x_image,1,32,name='conv1')
        # 卷积层2
        conv_out = conv_layer(conv1,32,64,name='conv2')
    else:
        conv1 = conv_layer(x_image,1,64,name='conv')
        conv_out = max_pool_2x2(conv1)

    size_in = 7*7*64
    flattened = tf.reshape(conv_out,[-1,size_in])
    
    if use_two_fc:
        # 全连接层
        fc1 = fc_layer(flattened, size_in, 512, name='fc1')
        embedding_input = fc1
        embedding_size = 512
        
        # Dropout
        keep_prob = tf.placeholder(tf.float32)
        h_fc1_drop = tf.nn.dropout(fc1, keep_prob)

        # Readout layer
        logits = fc_layer(h_fc1_drop,512,10, name='fc2')
    else:
        embedding_input = flattened
        embedding_size = size_in
        logits = fc_layer(flattened,size_in, 10, name='fc')
        
    # loss function 
    with tf.name_scope('loss'):
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=logits))
        tf.summary.scalar('loss', cross_entropy)

    with tf.name_scope('train'):
        optimizer = tf.train.AdamOptimizer(learning_rate)
        train_step = optimizer.minimize(cross_entropy)

    with tf.name_scope('accuracy'):
        corrent_prediction = tf.equal(tf.arg_max(logits,1),tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(corrent_prediction,tf.float32))
        tf.summary.scalar('accuracy', accuracy)
        
    summ = tf.summary.merge_all()
    
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(LOG_DIR+hparam, sess.graph)
    
    embedding = tf.Variable(tf.zeros([1024,embedding_size], name='test_embedding'))
    assignment = embedding.assign(embedding_input)
    config = tf.contrib.tensorboard.plugins.projector.ProjectorConfig()
    embedding_config = config.embeddings.add()
    embedding_config.tensor_name = embedding.name
    embedding_config.sprite.image_path = LOG_DIR+'sprite_1024.png'
    embedding_config.metadata_path = LOG_DIR+'labels_1024,tsv'
    embedding_config.sprite.single_image_dim.extend([28,28])
    tf.contrib.tensorboard.plugins.projector.visualize_embeddings(writer,config)
    
    batch_size = 50
    for i in range(2001):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        if i%5 == 0:
            train_accuracy,s = sess.run([accuracy,summ], feed_dict={x: batch_xs, y:batch_ys, keep_prob:1})
            writer.add_summary(s, i)
        if i%500 == 0:
            sess.run(assignment, feed_dict={x:mnist.test.images[:1024], y:mnist.test.labels[:1024]})
            saver.save(sess, os.path.join(LOG_DIR, 'model.ckpt'), i)
        sess.run(train_step, feed_dict={x: batch_xs, y:batch_ys, keep_prob:0.5})
        print('test accuracy:',sess.run(accuracy, feed_dict={x:mnist.test.images, mnist.test.labels, keep_prob:1}))

In [7]:
learning_rate = 1e-4
use_two_conv = True
use_two_fc = True

for lr in [1e-2,1e-4,1e-6]:
    hparam = make_hparam_string(lr,use_two_conv,use_two_fc)
    model(learning_rate,use_two_conv,use_two_fc,hparam)

In [8]:
for lr in [1e-2,1e-4,1e-6]:
    print(lr)

0.01
0.0001
1e-06
